Does out of sample performance vary with changing

the number of examples in Fine-Tuning
what is the smallest number that you think is sufficient
there are many choices of proper subsets of a given size
does it matter which one you choose ?

Continuation back in [Main Notebook](../part1_finetuning_Distilbert.ipynb)